In [3]:
# V1 without saving bm25 in same repo than spacy model
import copy
import spacy
import pickle
import numpy as np
class Scoring(object):
    def __init__(self, param):
        self.param = param
    def weights(self, document):
        return [self.param for token in document]
class ScorerComponent(object):
    name = "scorer_component"

    def __init__(self):
        self.data = {}
        self.IGNORED_POS = ['PRON', 'AUX', 'DET', "PUNCT"]

    def add_scorer(self, scorer):
        self.data["scorer"] = copy.deepcopy(scorer)

    # TODO: refactor
    def __call__(self, doc):
        " Score each non-punctuation token from spacy doc, and overwrite the vector representation using BM25 weights"
        word_tokens = [token.text for token in doc if token.pos_ not in self.IGNORED_POS]
        word_vectors = [token.vector for token in doc if token.pos_ not in self.IGNORED_POS]
        weights = self.data["scorer"].weights(word_tokens)
        doc.vector = np.average(word_vectors, weights=np.array(weights, dtype=np.float32),axis=0)
        return doc
    def to_disk(self, path, **kwargs):
        data_path = path / "words_scorer.pckl"
        print("Saving scorer to {}.".format(data_path))
        with open(data_path, "wb") as f:
            pickle.dump(self.data["scorer"], f)
    def from_disk(self, path, **kwargs):
        data_path = path / "words_scorer.pckl"
        print("Loading scorer.")
        with open(data_path, "rb") as f:
            self.data["scorer"] = pickle.load(f)

In [4]:
# loading back
from spacy.language import Language
Language.factories["scorer_component"] = lambda nlp, **cfg: ScorerComponent()
nlp_l = spacy.load("../../data/model/")
nlp_l.pipeline

Loading scorer.


[('tagger', <spacy.pipeline.pipes.Tagger at 0x1a428b2fcc8>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x1a44e39d3a8>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x1a44e39d4c8>),
 ('scorer_component', <__main__.ScorerComponent at 0x1a44e39a488>)]

In [5]:
nlp_l("est un test pollution marine").vector.sum()

1.3702624